<center><h1>AlexNet</h1></center>

<center><p><a href="https://proceedings.neurips.cc/paper_files/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html">ImageNet Classification with Deep Convolutional Neural Networks</a></p></center>

<img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-22_at_6.35.45_PM.png" width="1000"/>

In [1]:
import torch
from torch import nn

# AlexNet

In [2]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, init_weights=True, dropout=0.5):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),  # 224 -> 55
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # 55 -> 27
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # 27 -> 13
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # 13 -> 6
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

# Summary

## Data

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data = torch.randn((32, 3, 224, 224)).to(device)

## AlexNet

In [4]:
from torchkeras import summary

net = AlexNet(num_classes=1000).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                            [-1, 96, 55, 55]               34,944
ReLU-2                              [-1, 96, 55, 55]                    0
MaxPool2d-3                         [-1, 96, 27, 27]                    0
Conv2d-4                           [-1, 256, 27, 27]              614,656
ReLU-5                             [-1, 256, 27, 27]                    0
MaxPool2d-6                        [-1, 256, 13, 13]                    0
Conv2d-7                           [-1, 384, 13, 13]              885,120
ReLU-8                             [-1, 384, 13, 13]                    0
Conv2d-9                           [-1, 384, 13, 13]            1,327,488
ReLU-10                            [-1, 384, 13, 13]                    0
Conv2d-11                          [-1, 256, 13, 13]              884,992
ReLU-12                            [-